# CLL Morphology Analysis and Feature Selection
August 14, 2020

**Vladislav Kim**

Here we perform basic feature selection based on replicate correlation. Multiple replicates exist.

+ These plates are the same patient, screened in different plates on the same day:
    180306 Plate 5 and 180306 Plate 1
+ These plates are the same patient, but different aliquots, screened on different days:
    180306 Plate 3 and 180424 Plate 6
+ These plates are the same patient, screened in different plates on the same day:
    180213_Plate1 and 180213_Plate4
+ **Excluded due to low replicate correlation in terms of cell count (possibly segmentation errors)** These plates are the same patient, but different aliquots, screened on the same day:
    180504 Plate 6 and 180504 Plate 10

In [ ]:
import javabridge
import bioformats as bf
import skimage
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import os
import re
import sys
import h5py

javabridge.start_vm(class_path=bf.JARS)

In [ ]:
from bioimg import load_imgstack, load_image_series, read_bbox
from sklearn.preprocessing import label_binarize

def get_train_instance(path, fname, columns=['ymin','xmin','ymax','xmax'], pad=0):
    imgstack = load_imgstack(fname=os.path.join(path, fname + ".png"),
                            verbose=False)
    img = np.squeeze(imgstack)
    df = pd.read_csv(os.path.join(path, fname + ".csv"))
    rmax, cmax, _ = img.shape
    bbox = read_bbox(df=df, rmax=rmax,
                     cmax=cmax, columns=columns,
                     pad=pad)
    return img, bbox

## Check Bounding Boxes of CLL Nuclei

In [ ]:
path = '/Volumes/gitlab/microscopy/data/Sophie/Evaluations/'
plate = '180306_Plate1'

In [ ]:
for f in os.listdir(path):
    if re.search(plate, f):
        screen_id = f

path = path + screen_id

In [ ]:
df = pd.read_table(os.path.join(path, 'Evaluation5', 'Objects_Population - CLL cell nuclei.txt'), skiprows=9)

In [ ]:
# remove objects with area less than 10 um^2
df = df[df['CLL cell nuclei - CLL Nuclei Area [µm²]'] > 10].reset_index(drop=True)

In [ ]:
df['Row'] = df['Row'].astype(str)
df['Column'] = df['Column'].astype(str)
df['Field'] = df['Field'].astype(str)

In [ ]:
df = df.assign(well=lambda x: 'r' + x['Row'].str.zfill(2) + 'c' + x['Column'].str.zfill(2))

In [ ]:
df = df.assign(wellpos=lambda x: x['well'] + 'f' + x['Field'].str.zfill(2))

In [ ]:
df = df[['well','wellpos', 'Bounding Box', 'X', 'Y']]

In [ ]:
from ast import literal_eval
df[['xmin','ymin','xmax','ymax']] = pd.DataFrame(df['Bounding Box'].apply(lambda x: list(literal_eval(x))).tolist())

In [ ]:
df.head()

In [ ]:
all_wells = df['well'].unique()

In [ ]:
imgdir = '/Volumes/gitlab/microscopy/data/Sophie/CLL/'
imgdir = os.path.join(imgdir, screen_id, 'Images')

In [ ]:
fnames = [f for f in os.listdir(imgdir) if '.tiff' in f]
wfiles = [f for f in fnames if 'r01c02f01' in f and '(2)' not in f]

In [ ]:
imgstack = load_image_series(path=imgdir, imgfiles=[w for w in wfiles if 'ch1' in w])
hoechst = np.max(imgstack, axis=0)
imgstack = load_image_series(path=imgdir, imgfiles=[w for w in wfiles if 'ch2' in w])
ly = np.max(imgstack, axis=0)

In [ ]:
from bioimg import plot_channels
plot_channels([hoechst**0.5, ly**0.5],
              nrow=1, ncol=2,
              titles=['Nuclei', 'Lysosomes'])

In [ ]:
from bioimg import combine_channels
col_params = dict(colors=['blue', 'white'],
                               blend = [1.5, 0.7],
                               gamma=[0.5, 0.4])
img_overlay = combine_channels([hoechst, ly],
                               **col_params)

In [ ]:
well_df = df[df['wellpos']=='r01c02f01']

In [ ]:
gamma = 0.3
hoechst = hoechst**gamma
ly = ly**gamma

In [ ]:
rmax, cmax = hoechst.shape
bbox = read_bbox(df=well_df, rmax=rmax,
                 cmax=cmax, columns=['ymin','xmin','ymax','xmax'],
                 pad=5)

In [ ]:
from bioimg import show_bbox
show_bbox(img=img_overlay, bbox=bbox)

In [ ]:
from bioimg import ImgX
# initialize 'ImgX' class
imgx = ImgX(img=np.stack([hoechst, ly], axis=-1), 
            bbox=bbox,
            n_chan=['Hoechst', 'Lysosomal'])

In [ ]:
imgx.compute_props()
img_df = imgx.get_df().copy()

In [ ]:
img_df.shape

## Explore DMSO Wells of 180306_Plate1

In [ ]:
def load_CLL_cells(platedir, wells, annot, which=[1,2]):
    imgdf = []
    for w in wells:
        if os.path.isfile(os.path.join(platedir, w+'.csv')):
            df = pd.read_csv(os.path.join(platedir, w+'.csv'))
            df['well'] = w
            imgdf.append(df[np.isin(df['class'], which)])
    imgdf = pd.concat(imgdf).reset_index(drop=True)
    labels = imgdf[['class', 'well']]
    imgdf = imgdf.drop(['class', 'well'], axis=1)
    labels['class'] = labels['class'].apply(lambda x: 'Viable' if x == 2 else 'Apoptotic')    
    labels = pd.merge(labels, annot, on='well')
    return imgdf, labels

In [ ]:
# load plate annotation
annot_df = pd.read_csv('../data/AML_trainset/drugannot.txt',
                      sep='\t')

In [ ]:
annot_df.head()

In [ ]:
dmso = annot_df[annot_df.Drug == 'DMSO'].reset_index(drop=True)
dmso_wells = dmso['well'].unique()

In [ ]:
prefix = '../data/CLLdata/'

In [ ]:
ctrl_df, ctrl_annot = load_CLL_cells(platedir=os.path.join(prefix,'180306_Plate1'),
                            wells=dmso_wells, annot=dmso)

In [ ]:
ctrl_df.head()

In [ ]:
from bioimg.singlecell import select_features, preprocess_data
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=1e-8).fit(ctrl_df)

In [ ]:
imgdf = preprocess_data(df=ctrl_df, sel=sel, glog=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
from bioimg.singlecell import scale_data, check_data
scaler = StandardScaler().fit(X=imgdf)
imgdf_scaled = scale_data(imgdf, scaler=scaler)

In [ ]:
check_data(imgdf_scaled)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
pcs = PCA(n_components=20).fit_transform(imgdf_scaled)
X_tsne = TSNE(n_components=2, random_state=21, perplexity=50).fit_transform(pcs)

In [ ]:
from bioimg.singlecell import plot_dimred
X_df = pd.concat([pd.DataFrame(X_tsne, columns=['tsne1', 'tsne2']), ctrl_annot], axis=1)

plot_dimred(X_df, 
            hue='Culture',
            style='class',
            style_order=['Viable', 'Apoptotic'],
            title='DMSO control wells')
plt.legend(loc='lower right',
           bbox_to_anchor=(1.3,0.3))

In [ ]:
feat_subset = ['ch-Hoechst-area', 
               'ch-Hoechst-mean_intensity',
               'ch-Hoechst-perimeter',
               'ch-Hoechst-eccentricity',
               'ch-Hoechst-solidity',
               'ch-Lysosomal-mean_intensity']
Xfeat = imgdf_scaled.loc[:,feat_subset]
X_df = pd.concat([X_df, Xfeat], axis=1)

In [ ]:
from bioimg.singlecell import facet_dimred
facet_dimred(X_df, feat_subset=feat_subset,
            nrows=2, ncols=3)

## Replicate Plates Screened on the same Day
For feature selection load only viable nuclei:

In [ ]:
all_wells = annot_df['well'].values

In [ ]:
rep1_df, rep1_annot = load_CLL_cells(platedir=os.path.join(prefix,'180306_Plate1'),
                            wells=all_wells, annot=annot_df, which=2)

In [ ]:
rep1_df = preprocess_data(df=rep1_df, sel=sel, glog=True)
# center and scale by control wells
scaler = StandardScaler().fit(rep1_df[rep1_annot['Drug']=='DMSO'])
rep1_scaled = scale_data(rep1_df, scaler=scaler)

In [ ]:
rep2_df, rep2_annot = load_CLL_cells(platedir=os.path.join(prefix,'180306_Plate5'),
                            wells=all_wells, annot=annot_df, which=2)

In [ ]:
rep2_df = preprocess_data(df=rep2_df, sel=sel, glog=True)
# center and scale by control wells
scaler = StandardScaler().fit(rep2_df[rep2_annot['Drug']=='DMSO'])
rep2_scaled = scale_data(rep2_df, scaler=scaler)

In [ ]:
rep1_df.shape[1] == rep2_df.shape[1]

In [ ]:
rep1_scaled.shape[1] == rep2_scaled.shape[1]

In [ ]:
from bioimg.singlecell import aggregate_profiles
prof_rep1 = aggregate_profiles(rep1_scaled, rep1_annot)
prof_rep2 = aggregate_profiles(rep2_scaled, rep2_annot)

In [ ]:
# Make sure that the same wells are present in both replicates
prof_rep1 = prof_rep1[np.isin(prof_rep1['well'], prof_rep2['well'])]
prof_rep2 = prof_rep2[np.isin(prof_rep2['well'], prof_rep1['well'])]

In [ ]:
def get_repcor(prof1, prof2):
    repcor = prof1.sort_values(by='well').corrwith(prof2.sort_values(by='well'))
    return repcor

In [ ]:
repcor = get_repcor(prof_rep1, prof_rep2)

In [ ]:
biol_rep1 = []
biol_rep2 = []

biol_rep1.append(prof_rep1)
biol_rep2.append(prof_rep2)

## Replicate Plates Screened on Different Days
For feature selection load only viable nuclei:

In [ ]:
rep1_df, rep1_annot = load_CLL_cells(platedir=os.path.join(prefix,'180306_Plate3'),
                            wells=all_wells, annot=annot_df, which=2)

In [ ]:
rep1_df = preprocess_data(df=rep1_df, sel=sel, glog=True)
# center and scale by control wells
scaler = StandardScaler().fit(rep1_df[rep1_annot['Drug']=='DMSO'])
rep1_scaled = scale_data(rep1_df, scaler=scaler)

In [ ]:
rep2_df, rep2_annot = load_CLL_cells(platedir=os.path.join(prefix,'180424_Plate6'),
                            wells=all_wells, annot=annot_df, which=2)

In [ ]:
rep2_df = preprocess_data(df=rep2_df, sel=sel, glog=True)
# center and scale by control wells
scaler = StandardScaler().fit(rep2_df[rep2_annot['Drug']=='DMSO'])
rep2_scaled = scale_data(rep2_df, scaler=scaler)

In [ ]:
rep1_df.shape[1] == rep2_df.shape[1]

In [ ]:
rep1_scaled.shape[1] == rep2_scaled.shape[1]

In [ ]:
prof_rep1 = aggregate_profiles(rep1_scaled, rep1_annot)
prof_rep2 = aggregate_profiles(rep2_scaled, rep2_annot)

In [ ]:
# Make sure that the same wells are present in both replicates
prof_rep1 = prof_rep1[np.isin(prof_rep1['well'], prof_rep2['well'])]
prof_rep2 = prof_rep2[np.isin(prof_rep2['well'], prof_rep1['well'])]

In [ ]:
repcor = get_repcor(prof_rep1, prof_rep2)

In [ ]:
biol_rep1.append(prof_rep1)
biol_rep2.append(prof_rep2)

## Replicate Plates Screened on the Same Day (sample with low viability)
For feature selection load only viable nuclei:

In [ ]:
rep1_df, rep1_annot = load_CLL_cells(platedir=os.path.join(prefix,'180213_Plate1'),
                            wells=all_wells, annot=annot_df, which=2)

In [ ]:
rep1_df = preprocess_data(df=rep1_df, sel=sel, glog=True)
# center and scale by control wells
scaler = StandardScaler().fit(rep1_df[rep1_annot['Drug']=='DMSO'])
rep1_scaled = scale_data(rep1_df, scaler=scaler)

In [ ]:
rep2_df, rep2_annot = load_CLL_cells(platedir=os.path.join(prefix,'180213_Plate4'),
                            wells=all_wells, annot=annot_df, which=2)

In [ ]:
rep2_df = preprocess_data(df=rep2_df, sel=sel, glog=True)
# center and scale by control wells
scaler = StandardScaler().fit(rep2_df[rep2_annot['Drug']=='DMSO'])
rep2_scaled = scale_data(rep2_df, scaler=scaler)

In [ ]:
rep1_df.shape[1] == rep2_df.shape[1]

In [ ]:
rep1_scaled.shape[1] == rep2_scaled.shape[1]

In [ ]:
prof_rep1 = aggregate_profiles(rep1_scaled, rep1_annot)
prof_rep2 = aggregate_profiles(rep2_scaled, rep2_annot)

In [ ]:
# Make sure that the same wells are present in both replicates
prof_rep1 = prof_rep1[np.isin(prof_rep1['well'], prof_rep2['well'])]
prof_rep2 = prof_rep2[np.isin(prof_rep2['well'], prof_rep1['well'])]

In [ ]:
repcor = get_repcor(prof_rep1, prof_rep2)

In [ ]:
biol_rep1.append(prof_rep1)
biol_rep2.append(prof_rep2)

In [ ]:
repcor = get_repcor(pd.concat(biol_rep1).reset_index(drop=True), 
                    pd.concat(biol_rep2).reset_index(drop=True))

In [ ]:
highcor = repcor[repcor > 0.5]
highcor_df = pd.DataFrame({'feature': highcor.index, 'repcor': highcor.values})
highcor_df['channel'] = np.NaN
highcor_df.loc[highcor_df['feature'].str.contains('Lysosomal'),'channel'] = 'Lysosomal'
highcor_df.loc[highcor_df['feature'].str.contains('Hoechst'),'channel'] = 'Hoechst'

In [ ]:
def cm2inch(*tupl):
    inch = 2.54
    if isinstance(tupl[0], tuple):
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'size'   : 9}

matplotlib.rc('font', **font)

In [ ]:
colors = ["denim blue", "magenta"]
pal = sn.xkcd_palette(colors)

plt.figure(figsize=cm2inch(5,4))
sn.barplot(data=highcor_df.groupby('channel', 
                                   as_index=False).agg('count'), 
           y='channel', x='feature', palette=pal)
plt.xlabel('Number of features ($r>0.5$)')
plt.ylabel('')
sn.set(style='white')
sn.despine()
#plt.savefig('../figures/CLL-repcor-features.pdf', bbox_inches='tight')

In [ ]:
biol_rep1[0]['sample'] = biol_rep2[0]['sample'] = 'P0042'
biol_rep1[1]['sample'] = biol_rep2[1]['sample'] = 'P0487'
biol_rep1[2]['sample'] = biol_rep2[2]['sample'] = 'OMZP0134'

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
plt.plot([-3, 3], [-3, 3], linewidth=1.5, linestyle='--', color='black')
sn.scatterplot(x='ch-Lysosomal-area_x', y='ch-Lysosomal-area_y',
               data=pd.merge(pd.concat(biol_rep1)[['well', 'sample', 'ch-Lysosomal-area']],
                             pd.concat(biol_rep2)[['well', 'sample', 'ch-Lysosomal-area']],
                             on=['well', 'sample']), ax=ax,
              facecolor='#c20078')
sn.despine()
ax.annotate("r = {:.2f}".format(repcor['ch-Lysosomal-area']),
            xy=(.1, .9), xycoords=ax.transAxes)
ax.set_xlim((-2,3.5))
ax.set_ylim((-2,3.5))
plt.xlabel('Lysosomal area (rep 1)')
plt.ylabel('Lysosomal area (rep 2)')
xticks = ax.xaxis.get_major_ticks() 
xticks[0].label1.set_visible(False)
yticks = ax.yaxis.get_major_ticks() 
yticks[0].label1.set_visible(False)
sn.set(font_scale=1.4, style='white')

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
plt.plot([-3, 3], [-3, 3], linewidth=1.5, linestyle='--', color='black')
sn.scatterplot(x='ch-Lysosomal-contrast-3_x', y='ch-Lysosomal-contrast-3_y',
               data=pd.merge(pd.concat(biol_rep1)[['well', 'sample', 'ch-Lysosomal-contrast-3']],
                             pd.concat(biol_rep2)[['well', 'sample', 'ch-Lysosomal-contrast-3']],
                             on=['well', 'sample']), ax=ax,
              facecolor='#c20078')
sn.despine()
ax.annotate("r = {:.2f}".format(repcor['ch-Lysosomal-contrast-3']),
            xy=(.1, .9), xycoords=ax.transAxes)
ax.set_xlim((-3,3.5))
ax.set_ylim((-3,3.5))
plt.xlabel('Lysosomal contrast (rep 1)')
plt.ylabel('Lysosomal contrast (rep 2)')
xticks = ax.xaxis.get_major_ticks() 
xticks[0].label1.set_visible(False)
yticks = ax.yaxis.get_major_ticks() 
yticks[0].label1.set_visible(False)
sn.set(font_scale=1.4, style='white')

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
plt.plot([-3, 3], [-3, 3], linewidth=1.5, linestyle='--', color='black')
sn.scatterplot(x='ch-Lysosomal-mean_intensity_x', y='ch-Lysosomal-mean_intensity_y',
               data=pd.merge(pd.concat(biol_rep1)[['well', 'sample', 'ch-Lysosomal-mean_intensity']],
                             pd.concat(biol_rep2)[['well', 'sample', 'ch-Lysosomal-mean_intensity']],
                             on=['well', 'sample']), ax=ax,
              facecolor='#c20078')
sn.despine()
ax.annotate("r = {:.2f}".format(repcor['ch-Lysosomal-mean_intensity']),
            xy=(.1, .9), xycoords=ax.transAxes)
ax.set_xlim((-2,3.5))
ax.set_ylim((-2,3.5))
plt.xlabel('Lysosomal mean intensity (rep 1)')
plt.ylabel('Lysosomal mean intensity (rep 2)')
xticks = ax.xaxis.get_major_ticks() 
xticks[0].label1.set_visible(False)
yticks = ax.yaxis.get_major_ticks() 
yticks[0].label1.set_visible(False)
sn.set(font_scale=1.4, style='white')

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
plt.plot([-2, 2], [-2, 2], linewidth=1.5, linestyle='--', color='black')
sn.scatterplot(x='ch-Hoechst-eccentricity_x', y='ch-Hoechst-eccentricity_y',
               data=pd.merge(pd.concat(biol_rep1)[['well', 'sample', 'ch-Hoechst-eccentricity']],
                             pd.concat(biol_rep2)[['well', 'sample', 'ch-Hoechst-eccentricity']], 
                             on=['well', 'sample']), ax=ax)
sn.despine()
ax.annotate("r = {:.2f}".format(repcor['ch-Hoechst-eccentricity']),
            xy=(.1, .9), xycoords=ax.transAxes)
ax.set_xlim((-2.1,2))
ax.set_ylim((-2.1,2))
plt.xlabel('Hoechst eccentricity (rep 1)')
plt.ylabel('Hoechst eccentricity (rep 2)')
xticks = ax.xaxis.get_major_ticks() 
xticks[0].label1.set_visible(False)
yticks = ax.yaxis.get_major_ticks() 
yticks[0].label1.set_visible(False)
sn.set(font_scale=1.4, style='white')

In [ ]:
prof_rep1 = pd.concat(biol_rep1)
prof_rep2 = pd.concat(biol_rep2)

In [ ]:
sel_feats = repcor[repcor > 0.5].index.values

In [ ]:
# create a dictionary with various selected feature lists
featdict = dict()
featdict['repcor'] = sel_feats

In [ ]:
X_subset = imgdf_scaled[sel_feats]

In [ ]:
pcs = PCA(n_components=20).fit_transform(X_subset)
X_tsne = TSNE(n_components=2, random_state=21, perplexity=50).fit_transform(pcs)

In [ ]:
X_df = pd.concat([pd.DataFrame(X_tsne, columns=['tsne1', 'tsne2']), ctrl_annot], axis=1)

plot_dimred(X_df, 
            hue='Culture',
            style='class',
            style_order=['Viable', 'Apoptotic'],
            title='DMSO control wells')
plt.legend(loc='lower right',
           bbox_to_anchor=(1.3,0.3))

In [ ]:
prof_rep1 = prof_rep1[sel_feats]
prof_rep2 = prof_rep2[sel_feats]

In [ ]:
from bioimg.singlecell import select_residcor
sel_feats = select_residcor(prof1=prof_rep1, prof2=prof_rep2,
                            sel = ['ch-Hoechst-eccentricity', 
                                   'ch-Lysosomal-contrast-3'])

In [ ]:
print("Number of selected features: %d" % len(sel_feats))

In [ ]:
featdict['residcor'] = sel_feats

In [ ]:
from bioimg.singlecell import plot_heatmap
X_subset = imgdf_scaled[sel_feats]
featnames = [f.replace('ch-', '') for f in X_subset.columns]
# feature correlation
featcor = pd.DataFrame(np.corrcoef(X_subset.T),
                       index=featnames,
                       columns=featnames )
plot_heatmap(featcor, xticklabels=True, size=(12,14))
plt.show()

In [ ]:
feat_subset = ['ch-Hoechst-eccentricity', 
               'ch-Lysosomal-moments_hu-1',
               'ch-Hoechst-moments_hu-3',
               'ch-Hoechst-solidity',
               'ch-Hoechst-zernike-r20-1',
               'ch-Hoechst-InfoMeas1-d5-1',
               'ch-Lysosomal-zernike-r20-24',
               'ch-Lysosomal-area',
               'ch-Lysosomal-extent']
Xfeat = imgdf_scaled.loc[:,feat_subset]
X_df = pd.concat([X_df, Xfeat], axis=1)

In [ ]:
from bioimg.singlecell import facet_dimred
facet_dimred(X_df, feat_subset=feat_subset,
            nrows=3, ncols=3)

In [ ]:
featdict = {k : v if type(v)==list else v.tolist() for k,v in featdict.items()}

In [ ]:
import json
with open('CLL-featselect.json', 'w') as fp:
    json.dump(featdict, fp)